**Let’s move to Stage 2: Tool-Augmented Agent — where your agent goes from just talking to thinking, acting, and observing like a true reasoning system.**

🧩 Stage 2: Tool-Augmented Agent (Reason → Act → Observe)

🎯 Goal

Teach your LangGraph agent to:

- Call tools (like a calculator or Wikipedia search)

- Parse tool calls from LLM output

- Feed tool results back into the reasoning loop

This introduces the core agentic loop:
👉 Reason → Act → Observe → Reflect

🧠 Concepts Introduced

- Tool nodes: Represent capabilities (APIs, functions, etc.)

- Structured output (tool_calls) from LLM messages

- Dynamic routing: Deciding at runtime which node to execute next

- Observation updates: Feeding tool results back into the agent

In [1]:
from langgraph.graph import StateGraph, MessagesState, END, START
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import wikipedia
from dotenv import load_dotenv
load_dotenv("../../config/local.env")


c:\Users\ytiam\notebooks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
class State(MessagesState):
    next_node: str
    tool_args: dict

In [20]:

# 1️⃣ Define tools
@tool
def calculator(expression: str) -> str:
    """Evaluates a simple arithmetic expression."""
    print(f"Calculating: {expression}")
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

@tool
def wiki_search(query: str) -> str:
    """Searches Wikipedia and returns a short summary."""
    print(f"Searching Wikipedia for: {query}")
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception:
        return "No summary found."

tools = [calculator, wiki_search]

In [21]:

# 2️⃣ Define the LLM (tool-aware model)
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [22]:
from langchain_core.messages import ToolMessage, SystemMessage, HumanMessage

In [23]:
from langgraph.prebuilt import ToolNode # Pre-built node for executing tools

# Create a ToolNode instance. This node will automatically execute any tool calls 
# generated by an LLM that is bound to these tools.
tool_node = ToolNode(tools)

In [24]:
from langchain_core.runnables import RunnableConfig

In [25]:

# 3️⃣ Agent node — decides what to do next
def agent_node(state: State):
    system_prompt = SystemMessage(
    content=(
        "You are a helpful assistant. "
        "You can use tools when needed to answer the user's question. "
        "If you already have all the information you need, respond directly. "
        "When your answer is complete, do NOT call any more tools.\n\n"
        "Below is the user query.\n"

    )
)
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)

    return {"messages": messages + [response]}


def should_continue(state: State, config: RunnableConfig):
    # Get the list of messages from the current state.
    messages = state["messages"]
    # Get the last message, which is the response from the agent.
    last_message = messages[-1]
    
    if not last_message.tool_calls:
        # If there are no tool calls, we are done. So, we
        return "end"
    # Otherwise, if there are tool calls,
    else:
        # We need to execute the tool(s). So, we return "continue" to route to the tool execution node.
        return "continue"

In [26]:
# 5️⃣ Build graph
builder = StateGraph(State)
builder.add_node("agent", agent_node)
builder.add_node("tool", tool_node)

builder.add_edge(START, "agent")
builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        # If `should_continue` returns "continue", route to `music_tool_node`.
        "continue": "tool",
        # If `should_continue` returns "end", terminate the graph execution.
        "end": END,
    },
)
builder.add_edge("tool", "agent")

graph = builder.compile()

In [27]:
import uuid # Module for generating unique identifiers
thread_id = uuid.uuid4()

config = {"configurable": {"thread_id": thread_id}}

# 6️⃣ Run it
query = "What is the capital of France, and what is 12 * 7 and what is the age of Rahul Dravid as on 2025?"
result = graph.invoke({"messages": [{"role": "user", "content": query}]}, config=config)

# print("\n🤖 Final Answer:", result["messages"][-1].content)


Searching Wikipedia for: capital of France
Calculating: 12 * 7


In [28]:
print("\n🤖 Final Answer:", result["messages"][-1].content)


🤖 Final Answer: The capital of France is Paris. The result of \( 12 \times 7 \) is 84.

To determine the age of Rahul Dravid in 2025, we first note that he was born on January 11, 1973. So in 2025, he will turn 52 years old.


🧭 How It Works

- The agent node uses llm.bind_tools(tools) — this allows the model to decide if it needs a tool call.

- When a tool_call is detected, we route to the tool node.

- The tool node executes the function and sends an observation back.

- The graph loops back to the agent, which integrates the observation and produces the final answer.

✅ What You Learned

- How LangGraph enables tool integration through conditional node routing

- How the Reason–Act–Observe loop is implemented structurally

- The separation of thinking (agent) and doing (tool) phases